In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
# scale = 1/5 
# scale = 1/10
# scale = 1/20 
# scale = 1/40

In [3]:
# def tf_aug(X_in, scaler=scale):
#     X_out = np.zeros_like(X_in)
#     for channel in range(X_in.shape[-1]):
#         this_X = X_in[..., channel][np.newaxis, ..., np.newaxis]
#         scale = np.abs(np.max(this_X) - np.min(this_X)) * scaler
#         this_X = jitter(this_X, sigma=scale)
#         this_X = scaling(this_X, sigma=scale)
#         this_X = time_warp(this_X, sigma=scale)
#         this_X = window_warp(this_X, window_ratio=scale)
#         X_out[..., channel] = this_X[0, ..., 0]
#     return X_out


# def augment_this(X_in, y_in):
#     X_out = tf.numpy_function(tf_aug, [X_in], tf.float32)    
#     return X_out, y_in


In [4]:
X, y, site = load_data()

2022-06-27 16:21:33.490091: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from sklearn.utils import shuffle, resample

In [6]:
from sklearn.metrics import r2_score, median_absolute_error

In [7]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [10]:
n_runs = 10

In [11]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [12]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    return pd.DataFrame(result)

In [13]:
dfs = []

In [14]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[0])
        dfs.append(this_eval)
        one_df = pd.concat(dfs)
        one_df.to_csv("lenet_1.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

10/10 [==============================] - 0s 16ms/step - loss: 16.4171 - mean_squared_error: 16.4171 - rmse: 4.0518 - mean_absolute_error: 3.1503 - val_loss: 11.0862 - val_mean_squared_error: 11.0862 - val_rmse: 3.3296 - val_mean_absolute_error: 2.4902 - lr: 0.0010
Epoch 21/1000
10/10 [==============================] - 0s 18ms/step - loss: 16.0205 - mean_squared_error: 16.0205 - rmse: 4.0026 - mean_absolute_error: 3.1408 - val_loss: 9.9158 - val_mean_squared_error: 9.9158 - val_rmse: 3.1489 - val_mean_absolute_error: 2.4205 - lr: 0.0010
Epoch 22/1000
10/10 [==============================] - 0s 17ms/step - loss: 15.8850 - mean_squared_error: 15.8850 - rmse: 3.9856 - mean_absolute_error: 3.1520 - val_loss: 11.4888 - val_mean_squared_error: 11.4888 - val_rmse: 3.3895 - val_mean_absolute_error: 2.5236 - lr: 0.0010
Epoch 23/1000
10/10 [==============================] - 0s 17ms/step - loss: 16.3211 - mean_squared_error: 16.3211 - rmse: 4.0399 - mean_absolute_error: 3.2080 - val_loss: 13.6351 

10/10 [==============================] - 0s 16ms/step - loss: 13.5148 - mean_squared_error: 13.5148 - rmse: 3.6762 - mean_absolute_error: 2.8954 - val_loss: 10.0996 - val_mean_squared_error: 10.0996 - val_rmse: 3.1780 - val_mean_absolute_error: 2.4175 - lr: 0.0010
Epoch 50/1000
10/10 [==============================] - 0s 16ms/step - loss: 13.6715 - mean_squared_error: 13.6715 - rmse: 3.6975 - mean_absolute_error: 2.8686 - val_loss: 9.7435 - val_mean_squared_error: 9.7435 - val_rmse: 3.1215 - val_mean_absolute_error: 2.3694 - lr: 0.0010
Epoch 51/1000
10/10 [==============================] - 0s 16ms/step - loss: 13.5887 - mean_squared_error: 13.5887 - rmse: 3.6863 - mean_absolute_error: 2.8853 - val_loss: 10.9654 - val_mean_squared_error: 10.9654 - val_rmse: 3.3114 - val_mean_absolute_error: 2.5139 - lr: 0.0010
Epoch 52/1000
10/10 [==============================] - 0s 17ms/step - loss: 13.7544 - mean_squared_error: 13.7544 - rmse: 3.7087 - mean_absolute_error: 2.9157 - val_loss: 9.4345 -

Epoch 79/1000
10/10 [==============================] - 0s 16ms/step - loss: 11.0936 - mean_squared_error: 11.0936 - rmse: 3.3307 - mean_absolute_error: 2.5570 - val_loss: 8.2638 - val_mean_squared_error: 8.2638 - val_rmse: 2.8747 - val_mean_absolute_error: 2.1989 - lr: 0.0010
Epoch 80/1000
10/10 [==============================] - 0s 18ms/step - loss: 10.1890 - mean_squared_error: 10.1890 - rmse: 3.1920 - mean_absolute_error: 2.4569 - val_loss: 7.6095 - val_mean_squared_error: 7.6095 - val_rmse: 2.7585 - val_mean_absolute_error: 2.1279 - lr: 0.0010
Epoch 81/1000
10/10 [==============================] - 0s 17ms/step - loss: 10.1426 - mean_squared_error: 10.1426 - rmse: 3.1847 - mean_absolute_error: 2.4757 - val_loss: 8.3535 - val_mean_squared_error: 8.3535 - val_rmse: 2.8902 - val_mean_absolute_error: 2.2019 - lr: 0.0010
Epoch 82/1000
10/10 [==============================] - 0s 17ms/step - loss: 10.1947 - mean_squared_error: 10.1947 - rmse: 3.1929 - mean_absolute_error: 2.4474 - val_loss

Epoch 109/1000
10/10 [==============================] - 0s 17ms/step - loss: 9.5274 - mean_squared_error: 9.5274 - rmse: 3.0867 - mean_absolute_error: 2.3966 - val_loss: 7.7290 - val_mean_squared_error: 7.7290 - val_rmse: 2.7801 - val_mean_absolute_error: 2.1099 - lr: 0.0010
Epoch 110/1000
10/10 [==============================] - 0s 17ms/step - loss: 8.7280 - mean_squared_error: 8.7280 - rmse: 2.9543 - mean_absolute_error: 2.3167 - val_loss: 8.2898 - val_mean_squared_error: 8.2898 - val_rmse: 2.8792 - val_mean_absolute_error: 2.1880 - lr: 0.0010
Epoch 111/1000
10/10 [==============================] - 0s 17ms/step - loss: 9.1555 - mean_squared_error: 9.1555 - rmse: 3.0258 - mean_absolute_error: 2.3557 - val_loss: 7.6606 - val_mean_squared_error: 7.6606 - val_rmse: 2.7678 - val_mean_absolute_error: 2.0959 - lr: 0.0010
Epoch 112/1000
10/10 [==============================] - 0s 17ms/step - loss: 9.4474 - mean_squared_error: 9.4474 - rmse: 3.0737 - mean_absolute_error: 2.4151 - val_loss: 8.

...

Epoch 277/1000
10/10 [==============================] - 0s 17ms/step - loss: 7.7411 - mean_squared_error: 7.7411 - rmse: 2.7823 - mean_absolute_error: 2.1531 - val_loss: 6.8644 - val_mean_squared_error: 6.8644 - val_rmse: 2.6200 - val_mean_absolute_error: 1.9659 - lr: 1.2500e-04
Epoch 278/1000
10/10 [==============================] - 0s 18ms/step - loss: 7.9326 - mean_squared_error: 7.9326 - rmse: 2.8165 - mean_absolute_error: 2.1748 - val_loss: 6.6087 - val_mean_squared_error: 6.6087 - val_rmse: 2.5707 - val_mean_absolute_error: 1.9725 - lr: 1.2500e-04
Epoch 279/1000
10/10 [==============================] - 0s 17ms/step - loss: 8.1768 - mean_squared_error: 8.1768 - rmse: 2.8595 - mean_absolute_error: 2.2107 - val_loss: 7.0647 - val_mean_squared_error: 7.0647 - val_rmse: 2.6580 - val_mean_absolute_error: 1.9836 - lr: 1.2500e-04
Epoch 280/1000
10/10 [==============================] - 0s 17ms/step - loss: 7.3125 - mean_squared_error: 7.3125 - rmse: 2.7042 - mean_absolute_error: 2.1261 - 

10/10 [==============================] - 0s 19ms/step - loss: 8.2328 - mean_squared_error: 8.2328 - rmse: 2.8693 - mean_absolute_error: 2.2290 - val_loss: 6.6722 - val_mean_squared_error: 6.6722 - val_rmse: 2.5831 - val_mean_absolute_error: 1.9332 - lr: 6.2500e-05
Epoch 306/1000
10/10 [==============================] - 0s 18ms/step - loss: 8.1187 - mean_squared_error: 8.1187 - rmse: 2.8493 - mean_absolute_error: 2.1991 - val_loss: 6.4936 - val_mean_squared_error: 6.4936 - val_rmse: 2.5482 - val_mean_absolute_error: 1.9280 - lr: 6.2500e-05
Epoch 307/1000
10/10 [==============================] - 0s 18ms/step - loss: 7.9081 - mean_squared_error: 7.9081 - rmse: 2.8121 - mean_absolute_error: 2.1915 - val_loss: 6.5852 - val_mean_squared_error: 6.5852 - val_rmse: 2.5662 - val_mean_absolute_error: 1.9287 - lr: 6.2500e-05
Epoch 308/1000
10/10 [==============================] - 0s 20ms/step - loss: 7.8323 - mean_squared_error: 7.8323 - rmse: 2.7986 - mean_absolute_error: 2.1554 - val_loss: 6.591

10/10 [==============================] - 0s 18ms/step - loss: 8.0346 - mean_squared_error: 8.0346 - rmse: 2.8345 - mean_absolute_error: 2.2064 - val_loss: 6.6201 - val_mean_squared_error: 6.6201 - val_rmse: 2.5730 - val_mean_absolute_error: 1.9327 - lr: 3.1250e-05
Epoch 334/1000
10/10 [==============================] - 0s 18ms/step - loss: 8.2639 - mean_squared_error: 8.2639 - rmse: 2.8747 - mean_absolute_error: 2.2053 - val_loss: 6.7301 - val_mean_squared_error: 6.7301 - val_rmse: 2.5942 - val_mean_absolute_error: 1.9428 - lr: 3.1250e-05
Epoch 335/1000
10/10 [==============================] - 0s 18ms/step - loss: 8.1283 - mean_squared_error: 8.1283 - rmse: 2.8510 - mean_absolute_error: 2.1917 - val_loss: 6.7042 - val_mean_squared_error: 6.7042 - val_rmse: 2.5893 - val_mean_absolute_error: 1.9395 - lr: 3.1250e-05
Epoch 336/1000
10/10 [==============================] - 0s 18ms/step - loss: 7.6649 - mean_squared_error: 7.6649 - rmse: 2.7686 - mean_absolute_error: 2.1250 - val_loss: 6.635

10/10 [==============================] - 0s 19ms/step - loss: 7.7392 - mean_squared_error: 7.7392 - rmse: 2.7819 - mean_absolute_error: 2.1660 - val_loss: 6.5944 - val_mean_squared_error: 6.5944 - val_rmse: 2.5680 - val_mean_absolute_error: 1.9293 - lr: 1.5625e-05
Epoch 362/1000
10/10 [==============================] - 0s 19ms/step - loss: 8.3868 - mean_squared_error: 8.3868 - rmse: 2.8960 - mean_absolute_error: 2.2353 - val_loss: 6.6605 - val_mean_squared_error: 6.6605 - val_rmse: 2.5808 - val_mean_absolute_error: 1.9340 - lr: 1.5625e-05
Epoch 363/1000
10/10 [==============================] - 0s 21ms/step - loss: 7.5558 - mean_squared_error: 7.5558 - rmse: 2.7488 - mean_absolute_error: 2.0725 - val_loss: 6.6580 - val_mean_squared_error: 6.6580 - val_rmse: 2.5803 - val_mean_absolute_error: 1.9333 - lr: 1.5625e-05
Epoch 364/1000
10/10 [==============================] - 0s 17ms/step - loss: 7.8779 - mean_squared_error: 7.8779 - rmse: 2.8068 - mean_absolute_error: 2.1845 - val_loss: 6.694

Epoch 389/1000
10/10 [==============================] - 0s 18ms/step - loss: 7.6476 - mean_squared_error: 7.6476 - rmse: 2.7654 - mean_absolute_error: 2.1560 - val_loss: 6.6768 - val_mean_squared_error: 6.6768 - val_rmse: 2.5840 - val_mean_absolute_error: 1.9359 - lr: 3.9063e-06
Epoch 390/1000
10/10 [==============================] - 0s 17ms/step - loss: 7.5592 - mean_squared_error: 7.5592 - rmse: 2.7494 - mean_absolute_error: 2.1228 - val_loss: 6.6651 - val_mean_squared_error: 6.6651 - val_rmse: 2.5817 - val_mean_absolute_error: 1.9348 - lr: 3.9063e-06
Epoch 391/1000
10/10 [==============================] - 0s 17ms/step - loss: 7.8143 - mean_squared_error: 7.8143 - rmse: 2.7954 - mean_absolute_error: 2.1787 - val_loss: 6.6719 - val_mean_squared_error: 6.6719 - val_rmse: 2.5830 - val_mean_absolute_error: 1.9354 - lr: 3.9063e-06
Epoch 392/1000
10/10 [==============================] - 0s 17ms/step - loss: 7.8943 - mean_squared_error: 7.8943 - rmse: 2.8097 - mean_absolute_error: 2.2057 - 